In [2]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt

# train test split
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

from sklearn.model_selection import cross_validate

# encoding
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import category_encoders as ce

# scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler 

# column transformer & pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# cross validation
from sklearn.model_selection import cross_val_score

# algorithm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV,StratifiedKFold,train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier

# hyperparameter tuning
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


import shap

# Ignore Warning
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")  

c:\Users\tgs_e\anaconda3\envs\purwa\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Goals 

- Predict the 

In [3]:
df_test = pd.read_csv('D:/Machine Learning Learn/mentalHealth/test.csv')

df_train = pd.read_csv('D:/Machine Learning Learn/mentalHealth/train.csv')

In [4]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93800 entries, 0 to 93799
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     93800 non-null  int64  
 1   Name                                   93800 non-null  object 
 2   Gender                                 93800 non-null  object 
 3   Age                                    93800 non-null  float64
 4   City                                   93800 non-null  object 
 5   Working Professional or Student        93800 non-null  object 
 6   Profession                             69168 non-null  object 
 7   Academic Pressure                      18767 non-null  float64
 8   Work Pressure                          75022 non-null  float64
 9   CGPA                                   18766 non-null  float64
 10  Study Satisfaction                     18767 non-null  float64
 11  Jo

In [5]:
df_test

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,140700,Shivam,Male,53.0,Visakhapatnam,Working Professional,Judge,NaN,2.0,NaN,NaN,5.0,Less than 5 hours,Moderate,LLB,No,9.0,3.0,Yes
1,140701,Sanya,Female,58.0,Kolkata,Working Professional,Educational Consultant,NaN,2.0,NaN,NaN,4.0,Less than 5 hours,Moderate,B.Ed,No,6.0,4.0,No
2,140702,Yash,Male,53.0,Jaipur,Working Professional,Teacher,NaN,4.0,NaN,NaN,1.0,7-8 hours,Moderate,B.Arch,Yes,12.0,4.0,No
3,140703,Nalini,Female,23.0,Rajkot,Student,NaN,5.0,NaN,6.84,1.0,NaN,More than 8 hours,Moderate,BSc,Yes,10.0,4.0,No
4,140704,Shaurya,Male,47.0,Kalyan,Working Professional,Teacher,NaN,5.0,NaN,NaN,5.0,7-8 hours,Moderate,BCA,Yes,3.0,4.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93795,234495,Zoya,Female,49.0,Jaipur,Working Professional,Pilot,NaN,3.0,NaN,NaN,5.0,Less than 5 hours,Moderate,BSc,Yes,2.0,2.0,Yes
93796,234496,Shlok,Male,29.0,Ahmedabad,Working Professional,Pilot,NaN,5.0,NaN,NaN,1.0,7-8 hours,Moderate,BE,Yes,11.0,3.0,Yes
93797,234497,Rishi,Male,24.0,Visakhapatnam,Student,NaN,1.0,NaN,7.51,4.0,NaN,7-8 hours,Moderate,B.Tech,No,7.0,1.0,No
93798,234498,Eshita,Female,23.0,Kalyan,Working Professional,Marketing Manager,NaN,4.0,NaN,NaN,2.0,5-6 hours,Healthy,BA,Yes,7.0,5.0,Yes


In [6]:
pd.DataFrame({
    'column': df_train.columns.values,
    'datatype': df_train.dtypes.values,
    'nullvalue(%)': df_train.isna().mean().values * 100,
    'negativevalue(%)': [len(df_train[col][df_train[col] < 0]) / len(df_train) * 100 if col in df_train.select_dtypes(include=[np.number]).columns else 0 for col in df_train.columns],
    '0value(%)': [len(df_train[col][df_train[col] == 0]) / len(df_train) * 100 if col in df_train.select_dtypes(include=[np.number]).columns else 0 for col in df_train.columns],
    'duplicate' : df_train.duplicated().sum(),
    'uniquevalue': df_train.nunique().values,
    'sample_unique': [df_train[col].unique() for col in df_train.columns]}
).round(3)

,column,datatype,nullvalue(%),negativevalue(%),0value(%),duplicate,uniquevalue,sample_unique
0,id,int64,0.000,0.0,0.001,0,140700,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,Name,object,0.000,0.0,0.000,0,422,"[Aaradhya, Vivan, Yuvraj, Rhea, Vani, Ritvik, ..."
2,Gender,object,0.000,0.0,0.000,0,2,"[Female, Male]"
3,Age,float64,0.000,0.0,0.000,0,43,"[49.0, 26.0, 33.0, 22.0, 30.0, 59.0, 47.0, 38...."
4,City,object,0.000,0.0,0.000,0,98,"[Ludhiana, Varanasi, Visakhapatnam, Mumbai, Ka..."
5,Working Professional or Student,object,0.000,0.0,0.000,0,2,"[Working Professional, Student]"
6,Profession,object,26.034,0.0,0.000,0,64,"[Chef, Teacher, nan, Business Analyst, Finanan..."
7,Academic Pressure,float64,80.173,0.0,0.000,0,5,"[nan, 5.0, 2.0, 3.0, 4.0, 1.0]"
8,Work Pressure,float64,19.842,0.0,0.000,0,5,"[5.0, 4.0, nan, 1.0, 2.0, 3.0]"
9,CGPA,float64,80.172,0.0,0.000,0,331,"[nan, 8.97, 5.9, 7.03, 5.59, 8.13, 5.7, 9.54, ..."


In [7]:
pd.DataFrame({
    'column': df_test.columns.values,
    'datatype': df_test.dtypes.values,
    'nullvalue(%)': df_test.isna().mean().values * 100,
    'negativevalue(%)': [len(df_test[col][df_test[col] < 0]) / len(df_test) * 100 if col in df_test.select_dtypes(include=[np.number]).columns else 0 for col in df_test.columns],
    '0value(%)': [len(df_test[col][df_test[col] == 0]) / len(df_test) * 100 if col in df_test.select_dtypes(include=[np.number]).columns else 0 for col in df_test.columns],
    'duplicate' : df_test.duplicated().sum(),
    'uniquevalue': df_test.nunique().values,
    'sample_unique': [df_test[col].unique() for col in df_test.columns]}
).round(3)

,column,datatype,nullvalue(%),negativevalue(%),0value(%),duplicate,uniquevalue,sample_unique
0,id,int64,0.000,0.0,0.00,0,93800,"[140700, 140701, 140702, 140703, 140704, 14070..."
1,Name,object,0.000,0.0,0.00,0,374,"[Shivam, Sanya, Yash, Nalini, Shaurya, Kartik,..."
2,Gender,object,0.000,0.0,0.00,0,2,"[Male, Female]"
3,Age,float64,0.000,0.0,0.00,0,44,"[53.0, 58.0, 23.0, 47.0, 29.0, 28.0, 21.0, 45...."
4,City,object,0.000,0.0,0.00,0,68,"[Visakhapatnam, Kolkata, Jaipur, Rajkot, Kalya..."
5,Working Professional or Student,object,0.000,0.0,0.00,0,2,"[Working Professional, Student]"
6,Profession,object,26.260,0.0,0.00,0,64,"[Judge, Educational Consultant, Teacher, nan, ..."
7,Academic Pressure,float64,79.993,0.0,0.00,0,5,"[nan, 5.0, 1.0, 3.0, 4.0, 2.0]"
8,Work Pressure,float64,20.019,0.0,0.00,0,5,"[2.0, 4.0, nan, 5.0, 1.0, 3.0]"
9,CGPA,float64,79.994,0.0,0.00,0,326,"[nan, 6.84, 7.39, 6.03, 8.63, 8.61, 6.73, 5.58..."


## Combine academic pressure with work pressure

In [12]:
df_test['Pressure'] = df_test['Academic Pressure'].combine_first(df_test['Work Pressure'])
df_train['Pressure'] = df_train['Academic Pressure'].combine_first(df_train['Work Pressure'])
df_test.drop(['Academic Pressure', 'Work Pressure'], axis=1, inplace=True)
df_train.drop(['Academic Pressure', 'Work Pressure'], axis=1, inplace=True)

In [13]:
df_train.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression,Pressure
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0,5.0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1,4.0
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1,5.0
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1,5.0
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0,1.0


In [14]:
df_test['Satisfaction'] = df_test['Study Satisfaction'].combine_first(df_test['Job Satisfaction'])
df_train['Satisfaction'] = df_train['Study Satisfaction'].combine_first(df_train['Job Satisfaction'])
df_test.drop(['Study Satisfaction', 'Job Satisfaction'], axis=1, inplace=True)
df_train.drop(['Study Satisfaction', 'Job Satisfaction'], axis=1, inplace=True)

In [16]:
df_train.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,CGPA,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression,Pressure,Satisfaction
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0,5.0,2.0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1,4.0,3.0
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,8.97,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1,5.0,2.0
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1,5.0,1.0
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0,1.0,1.0


In [17]:
df_test.drop(['Name'], axis=1, inplace=True)
df_train.drop(['Name',], axis=1, inplace=True)

In [18]:
df_test.head()

,id,Gender,Age,City,Working Professional or Student,Profession,CGPA,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Pressure,Satisfaction
0,140700,Male,53.0,Visakhapatnam,Working Professional,Judge,NaN,Less than 5 hours,Moderate,LLB,No,9.0,3.0,Yes,2.0,5.0
1,140701,Female,58.0,Kolkata,Working Professional,Educational Consultant,NaN,Less than 5 hours,Moderate,B.Ed,No,6.0,4.0,No,2.0,4.0
2,140702,Male,53.0,Jaipur,Working Professional,Teacher,NaN,7-8 hours,Moderate,B.Arch,Yes,12.0,4.0,No,4.0,1.0
3,140703,Female,23.0,Rajkot,Student,NaN,6.84,More than 8 hours,Moderate,BSc,Yes,10.0,4.0,No,5.0,1.0
4,140704,Male,47.0,Kalyan,Working Professional,Teacher,NaN,7-8 hours,Moderate,BCA,Yes,3.0,4.0,No,5.0,5.0


In [19]:
df_test.drop(['CGPA'], axis=1, inplace=True)
df_train.drop(['CGPA',], axis=1, inplace=True)

In [20]:
df_test.head()

,id,Gender,Age,City,Working Professional or Student,Profession,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Pressure,Satisfaction
0,140700,Male,53.0,Visakhapatnam,Working Professional,Judge,Less than 5 hours,Moderate,LLB,No,9.0,3.0,Yes,2.0,5.0
1,140701,Female,58.0,Kolkata,Working Professional,Educational Consultant,Less than 5 hours,Moderate,B.Ed,No,6.0,4.0,No,2.0,4.0
2,140702,Male,53.0,Jaipur,Working Professional,Teacher,7-8 hours,Moderate,B.Arch,Yes,12.0,4.0,No,4.0,1.0
3,140703,Female,23.0,Rajkot,Student,NaN,More than 8 hours,Moderate,BSc,Yes,10.0,4.0,No,5.0,1.0
4,140704,Male,47.0,Kalyan,Working Professional,Teacher,7-8 hours,Moderate,BCA,Yes,3.0,4.0,No,5.0,5.0


In [21]:
pd.DataFrame({
    'column': df_train.columns.values,
    'datatype': df_train.dtypes.values,
    'nullvalue(%)': df_train.isna().mean().values * 100,
    'negativevalue(%)': [len(df_train[col][df_train[col] < 0]) / len(df_train) * 100 if col in df_train.select_dtypes(include=[np.number]).columns else 0 for col in df_train.columns],
    '0value(%)': [len(df_train[col][df_train[col] == 0]) / len(df_train) * 100 if col in df_train.select_dtypes(include=[np.number]).columns else 0 for col in df_train.columns],
    'duplicate' : df_train.duplicated().sum(),
    'uniquevalue': df_train.nunique().values,
    'sample_unique': [df_train[col].unique() for col in df_train.columns]}
).round(3)

,column,datatype,nullvalue(%),negativevalue(%),0value(%),duplicate,uniquevalue,sample_unique
0,id,int64,0.000,0.0,0.001,0,140700,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,Gender,object,0.000,0.0,0.000,0,2,"[Female, Male]"
2,Age,float64,0.000,0.0,0.000,0,43,"[49.0, 26.0, 33.0, 22.0, 30.0, 59.0, 47.0, 38...."
3,City,object,0.000,0.0,0.000,0,98,"[Ludhiana, Varanasi, Visakhapatnam, Mumbai, Ka..."
4,Working Professional or Student,object,0.000,0.0,0.000,0,2,"[Working Professional, Student]"
5,Profession,object,26.034,0.0,0.000,0,64,"[Chef, Teacher, nan, Business Analyst, Finanan..."
6,Sleep Duration,object,0.000,0.0,0.000,0,36,"[More than 8 hours, Less than 5 hours, 5-6 hou..."
7,Dietary Habits,object,0.003,0.0,0.000,0,23,"[Healthy, Unhealthy, Moderate, Yes, Pratham, B..."
8,Degree,object,0.001,0.0,0.000,0,115,"[BHM, LLB, B.Pharm, BBA, MCA, MD, BSc, ME, B.A..."
9,Have you ever had suicidal thoughts ?,object,0.000,0.0,0.000,0,2,"[No, Yes]"
